In [170]:
#import all modules here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [171]:
N = 14
df = pd.read_csv('../SPECT.train',header=None)
features = df.loc[:,df.columns!=0]
label = df[0]
label_column = 0

In [172]:
data_df = pd.read_csv("../haberman.data",header=None)
data_df.fillna(data_df.median(),inplace=True)
data_df.columns = ['Age','Year','Nodes','Survival_status']
data_df['Survival_status'] = data_df['Survival_status'].astype(int)
data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
label_column = 'Survival_status'

In [173]:
def add_noise(data, col):
    class0 = data.loc[data[col]!=1,:]
    class1 = data.loc[data[col]!=0,:]
    def noise(data):
        sample = data.sample(frac = 0.4)
        for i in list(sample.index):
            data[col][i] = int(not data[col][i])
        return list(sample.index)    
    noise0 = noise(class0)
    noise1 = noise(class1)
    noisy_indices = noise0+noise1
    mislabeled = pd.concat([class0,class1])
    indices = list(mislabeled.index)
    print(len(noisy_indices))
    return mislabeled

In [174]:
mislabeled = add_noise(data_df, label_column)

122


In [175]:
def distance_from_centroid(df,centroids,data_label):
    d = defaultdict(list)
    for index in df.index:
        dist = 0
        for i in range(len(centroids[0])):
            dist+=(centroids[df.loc[index,'cluster']][i] - df.loc[index,df.columns[i]])**2
        d[df.loc[index,'cluster']].append([dist,index,data_label[index]])
    return d

In [176]:
def ratio_in_cluster(features,data_label,n):
    majority_label_in_cluster = defaultdict(None)
    km = KMeans(n_clusters = n)
    km.fit(features)
    label = km.labels_
    print(len(label))
    print(features.shape)
    features['cluster'] = label
    
    distance = distance_from_centroid(features,km.cluster_centers_,data_label)
    
    for i in distance.keys():
        distance[i] = sorted(distance[i],key=lambda x: x[0])
        
    normalized_distance = dict(distance)
    label_ratio_count = {}
    
    for i in normalized_distance:
        temp = [0,0]
        for j in range(len(normalized_distance[i])):
            normalized_distance[i][j][0] = normalized_distance[i][j][0]/normalized_distance[i][-1][0]
            temp[normalized_distance[i][j][-1]] +=1

        temp[0],temp[1] = temp[0]/sum(temp),temp[1]/sum(temp)
        label_ratio_count[i] = temp
        
        if temp[0] > temp[1] and temp[0]>=0.7:
            majority_label_in_cluster[i] = 0
        elif temp[0] < temp[1] and temp[1]>=0.7:
            majority_label_in_cluster[i] = 1
        else:
            majority_label_in_cluster[i] = None

    return(normalized_distance, majority_label_in_cluster)
    #return pd.DataFrame(data=label_ratio_count),majority_label_in_cluster  
    

In [177]:
normalized_distance, majority_label_in_cluster = ratio_in_cluster(mislabeled.loc[:,mislabeled.columns!=label_column],mislabeled[label_column],14)

306
(306, 3)


In [178]:
points_removed = []
reduced_points = defaultdict(list)
for cluster_id in normalized_distance.keys():
    count = 0
    for point in normalized_distance[cluster_id]:
        if count/len(normalized_distance[cluster_id]) > 0.75:
            if point[-1] != majority_label_in_cluster[cluster_id] and majority_label_in_cluster[cluster_id]!=None:
                points_removed.append(point)
                continue
            else:
                reduced_points[cluster_id].append(point)
                count+=1
        else:
            reduced_points[cluster_id].append(point)
            count +=1

In [179]:
# result = []
# for cluster_id in normalized_distance:
#     print(len(normalized_distance[cluster_id]), len(reduced_points[cluster_id]))
#     print((len(reduced_points[cluster_id]) * 1.0) -  len(normalized_distance[cluster_id]))
#     print()

In [180]:
reduced_points

defaultdict(list,
            {0: [[0.019504480759093078, 16, 1],
              [0.03237967444609534, 18, 1],
              [0.04921646619063662, 32, 1],
              [0.0625868596348308, 26, 1],
              [0.0625868596348308, 27, 1],
              [0.07843325186498643, 19, 1],
              [0.07843325186498643, 23, 1],
              [0.11012603632529766, 15, 1],
              [0.1126020351112591, 10, 1],
              [0.1126020351112609, 25, 1],
              [0.11458283414003041, 28, 0],
              [0.1259724285554533, 13, 1],
              [0.15518921422980492, 20, 1],
              [0.16955000718838475, 35, 1],
              [0.1720260059743444, 7, 0],
              [0.1720260059743462, 29, 1],
              [0.17351160524592377, 38, 1],
              [0.2185747831504274, 6, 0],
              [0.24630596955320322, 37, 1],
              [0.30028274308717234, 39, 0],
              [0.3379179246337934, 41, 0],
              [0.3418795226913324, 40, 1],
              [0.38347

In [181]:
majority_label_in_cluster

defaultdict(None,
            {0: None,
             5: 0,
             12: None,
             4: None,
             7: 0,
             13: 1,
             6: None,
             1: None,
             3: None,
             10: None,
             9: None,
             2: None,
             8: None,
             11: None})

In [182]:
points_removed

[[0.9862885634153944, 4, 1], [0.9634931997136724, 74, 0]]